In [19]:
import pandas as pd 
import numpy as np 
import os
import kaleido
import joblib
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split,StratifiedKFold

from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn import metrics
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from config.path_config import ROOT_DIR
import plotly.io as pio
import matplotlib.pyplot as plt
CONFIG_PATH = ""
TARGET = "SeriousDlqin2yrs"

TRAINING_PATH = os.path.join(ROOT_DIR,"data","processed","processed_training_5_exp_tuned.csv")
submission_path = os.path.join(ROOT_DIR,"data","raw","sampleEntry.csv")
TEST_PATH = os.path.join(ROOT_DIR,"data","processed","processed_5_forth_exp_tuned.csv")
WRANGLED_DATA = os.path.join(ROOT_DIR,"data","interim","interim_training_forth_exp_tuned.csv")

MODEL_PATH = os.path.join(ROOT_DIR,"models","LGBMClassifier_exp_class_weight.joblib")

In [29]:
test_data = pd.read_csv(TEST_PATH)
submission = pd.read_csv(submission_path)
model = joblib.load(MODEL_PATH)
data = pd.read_csv(TRAINING_PATH)
#split into X and Y 
X = data.drop(TARGET,axis=1)
y = data[TARGET]

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2)

In [30]:
latest_params = {'lambda_l1': 0.0004979634231918787, 'lambda_l2': 8.888178821370811, 'num_leaves': 48, 'max_depth': 29, 'feature_fraction': 0.6437564293684458, 'bagging_fraction': 0.8402914515480595, 'bagging_freq': 9, 'min_child_samples': 115}
clf_model = LGBMClassifier(**latest_params,class_weight='balanced')
lgb_isotonic = CalibratedClassifierCV(clf_model, cv=5, method="isotonic",ensemble=True)
lgb_isotonic.fit(X,y)

[LightGBM] [Warning] lambda_l1 is set=0.0004979634231918787, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004979634231918787
[LightGBM] [Warning] feature_fraction is set=0.6437564293684458, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6437564293684458
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] lambda_l2 is set=8.888178821370811, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.888178821370811
[LightGBM] [Warning] bagging_fraction is set=0.8402914515480595, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8402914515480595
[LightGBM] [Warning] lambda_l1 is set=0.0004979634231918787, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004979634231918787
[LightGBM] [Warning] feature_fraction is set=0.6437564293684458, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6437564293684458
[LightGBM] [Warning] baggin

CalibratedClassifierCV(cv=5,
                       estimator=LGBMClassifier(bagging_fraction=0.8402914515480595,
                                                bagging_freq=9,
                                                class_weight='balanced',
                                                feature_fraction=0.6437564293684458,
                                                lambda_l1=0.0004979634231918787,
                                                lambda_l2=8.888178821370811,
                                                max_depth=29,
                                                min_child_samples=115,
                                                num_leaves=48),
                       method='isotonic')

In [31]:
pred = lgb_isotonic.predict_proba(test_data)
submission['Probability'] = pred[:, 1]
submission.to_csv('submission_weighted_class.csv',index=False)